<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/10e_Multi_output_CNN_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-output CNN Models**

**Import libraries**

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.vis_utils import plot_model
from keras.models import Sequential


**Deefine the split_sequences function**

In [ ]:
def split_sequences(sequences, n_steps): 
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences)-1: break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :] 
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

**Create the time series**

In [ ]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105, 115, 125]) 
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure 
in_seq1 = in_seq1.reshape((len(in_seq1), 1)) 
in_seq2 = in_seq2.reshape((len(in_seq2), 1)) 
out_seq = out_seq.reshape((len(out_seq), 1)) 

In [ ]:
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3

In [ ]:
dataset

In [ ]:
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# separate output
y1 = y[:, 0].reshape((y.shape[0], 1)) 
y2 = y[:, 1].reshape((y.shape[0], 1)) 
y3 = y[:, 2].reshape((y.shape[0], 1))

In [ ]:
for i in range(len(X)):
  print("X:\n",X[i], "y1:",y1[i], "y2:", y2[i], "y3:", y3[i])

**Create the model**

In [ ]:
from tensorflow.keras.layers import LeakyReLU
visible = Input(shape=(n_steps, n_features))
cnn = Conv1D(64, 2, activation='LeakyReLU')(visible)
cnn = MaxPooling1D()(cnn)
cnn = Flatten()(cnn)
cnn = Dense(50, activation='LeakyReLU')(cnn)

**Create the 3 different outputs for the model**

In [ ]:
# define output 1
output1 = Dense(1)(cnn)
# define output 2
output2 = Dense(1)(cnn)
# define output 3
output3 = Dense(1)(cnn)
model = Model(inputs=visible, outputs=[output1, output2, output3]) 

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

**Train the model**

In [ ]:
model.fit(X, [y1,y2,y3], epochs=2000, verbose=0)

**Make a prediction**<br>
Expecting [130, 135,265]

In [ ]:
x_input = array([[[100,105,205],[110,115,225],[120,125,245]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)